In [1]:
from pipeline.util.platform import start_spark, start_spark_local
from pipeline.util.storage import read_csv, read_parquet, write_parquet, write_repart_parquet, write_delta, write_deltatable
from pyspark.sql import functions as F
from pyspark.sql import types as T
from delta import *
import time

In [2]:
spark, loggrer, conf = start_spark(app_name='addessmatch_etl_job')

local


In [3]:
df_ab = read_parquet(spark, "{storage_curated}/data/in/addressbase".format(storage_curated = spark.conf.get('storage.curated')))

In [79]:
df_ab_id = df_ab.withColumn("ADDRESS_ID", F.monotonically_increasing_id() + F.unix_timestamp(F.current_timestamp()))

In [80]:
df_ab_id.createOrReplaceTempView("address_in")

In [82]:
loc = "'{storage_curated}/data/out/address'".format(storage_curated = spark.conf.get('storage.curated'))

#spark.sql(f"DROP TABLE IF EXISTS ADDRESS")
spark.sql(f"CREATE OR REPLACE TABLE ADDRESS ( \
          UPRN VARCHAR(12), \
          OUTCODE STRING, \
          LOCALITY STRING, \
          ADDRESS STRING, \
          ADDRESS_ID BIGINT )  \
          USING DELTA  \
          LOCATION {loc}")

DataFrame[]

In [84]:
# spark.sql(f"INSERT INTO ADDRESS ( \
#           UPRN,OUTCODE,LOCALITY,ADDRESS,ADDRESS_ID )  \
#           SELECT \
#           UPRN,OUTCODE,LOCALITY,ADDRESS,ADDRESS_ID \
#           FROM address_in")

DataFrame[]

In [102]:
spark.sql(f"MERGE INTO ADDRESS \
          USING address_in \
          ON ADDRESS.UPRN = address_in.UPRN \
          WHEN MATCHED THEN \
            UPDATE SET \
              OUTCODE = address_in.OUTCODE, \
              LOCALITY = address_in.LOCALITY, \
              ADDRESS = address_in.ADDRESS \
          WHEN NOT MATCHED THEN \
            INSERT ( \
              UPRN, OUTCODE, LOCALITY, ADDRESS, ADDRESS_ID \
            ) VALUES ( \
              address_in.UPRN, address_in.OUTCODE, address_in.LOCALITY, address_in.ADDRESS, address_in.ADDRESS_ID \
            )")

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [14]:
#write_delta(spark, df_abmatch, "{storage_curated}/data/out/address".format(storage_curated = spark.conf.get('storage.curated')))

In [ ]:
# DeltaTable.createOrReplace(spark) \
#     .tableName("address") \
#     .location("{storage_curated}/data/out/address".format(storage_curated = spark.conf.get('storage.curated'))) \
#     .addColumn("UPRN", "VARCHAR(12)") \
#     .addColumn("OUTCODE", "STRING") \
#     .addColumn("LOCALITY", "STRING") \
#     .addColumn("ADDRESS", "STRING") \
#     .addColumn("ADDRESS_ID", "BIGINT", generatedAlwaysAs=F.monotonically_increasing_id()) \
#     .execute()

In [7]:
#DeltaTable.isDeltaTable(spark, "{storage_curated}/data/out/address".format(storage_curated = spark.conf.get('storage.curated'))) # True
#spark.sql("show tables in default").show(truncate=False)
#spark.sql("describe table extended address_in").show(truncate=False)
#spark.sql("select * from address_in").show(truncate=False)
#spark.sql("select count(distinct OUTCODE) from ADDRESS").show(truncate=False)
#spark.sql("select count(*) from address").show()
#spark.sql("USE hive_metastore")
#spark.sql("show databases").show(truncate=False)
#spark.sql("show tables").show(truncate=False)
#spark.sql("drop table address")
#spark.sql("delete from address")
#print(spark.sharedState.externalCatalog.unwrapped)
#spark.catalog.listTables()
#spark.catalog.currentDatabase()
#spark.sparkContext.getConf().getAll()
#df_ab_id.printSchema()
#df_ab_id.show(truncate=False)
#df_ab.printSchema()

+-----------------------+
|count(DISTINCT OUTCODE)|
+-----------------------+
|1982                   |
+-----------------------+

